In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

2024-11-07 21:25:06.196150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 21:25:06.196190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 21:25:06.197742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 21:25:06.206961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
outflows = pd.read_parquet('outflows.pqt')
outflows.posted_date = pd.to_datetime(outflows.posted_date, format='%Y-%m-%d')

categories = ['GENERAL_MERCHANDISE', 'FOOD_AND_BEVERAGES', 'GROCERIES', 'TRAVEL', 'PETS', 'MORTGAGE', 'OVERDRAFT', 'EDUCATION', 'RENT']
outflows = outflows[outflows.category.isin(categories)]

In [3]:
outflows['new_memo'] = outflows.memo.str.lower().str.replace(r'\*(\S)', r' \1', regex=True)     # Remove all '*' but gotta check if there's any alphanumerics right after it
outflows['new_memo'] = outflows.new_memo.str.replace(r'[^a-z0-9]', ' ', regex=True)             # Remove all special chars
outflows['new_memo'] = outflows.new_memo.str.replace(r'x{3,}', ' ', regex=True)                 # Remove all triple X's
outflows['new_memo'] = outflows.new_memo.str.replace(r'withdrawal', 'withdrawal ', regex=True)  # Fixing the withdrawal rows
outflows['new_memo'] = outflows.new_memo.str.replace(r'\s+', ' ', regex=True).str.strip()       # Remove any extra whitespace

outflows = outflows[['prism_consumer_id', 'prism_account_id', 'memo', 'new_memo', 'amount',	'posted_date', 'category']]
# outflows[['memo', 'new_memo']]

In [4]:
outflows.drop(columns='memo', inplace=True)

## LLM:
    - RoBERTa:

In [5]:
# subset = outflows.iloc[:10000]

In [6]:
np.random.seed(42069)

# Splitting based on prism_consumer_id:
cids = outflows.prism_consumer_id.unique()
train_cids, test_cids = train_test_split(cids, test_size=0.3, random_state=420) # 70 / 30 split

label_encoder = LabelEncoder()
outflows['category_encoded'] = label_encoder.fit_transform(outflows.category)

# Features and labels for train and test sets:
X_train = outflows[outflows.prism_consumer_id.isin(train_cids)]
X_test  = outflows[outflows.prism_consumer_id.isin(test_cids)]

y_train = X_train.category_encoded
y_test  = X_test.category_encoded

len(X_train), len(X_test)

(922256, 384196)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model     = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=9)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
train_data = Dataset.from_dict({"text": X_train.new_memo, "label": y_train})
test_data = Dataset.from_dict({"text": X_test.new_memo, "label": y_test})

train_data = train_data.map(tokenize_data, batched=True)
test_data = test_data.map(tokenize_data, batched=True)

Map:   0%|          | 0/922256 [00:00<?, ? examples/s]

Map:   0%|          | 0/384196 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",        # Still evaluate every epoch, if desired
    save_strategy="no",           # Disable checkpoint saving
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)


/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

trainer.compute_metrics = compute_metrics
trainer.evaluate()

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.193136215209961,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.34681256442024383,
 'eval_f1': 0.1942345353899732,
 'eval_precision': 0.13489036077377844,
 'eval_recall': 0.34681256442024383,
 'eval_runtime': 8616.9834,
 'eval_samples_per_second': 44.586,
 'eval_steps_per_second': 5.573}